In [34]:
# Auxiliar imports
import sys, os, time, importlib
import matplotlib.pyplot as plt
import numpy as np

# Gym imports
import gym
from gym.vector import SyncVectorEnv

# PyTorch imports
import torch
from torch import nn, optim

# Custom imports
sys.path.append(os.path.abspath('..')) # Add parent directory to path

import ppo_network
importlib.reload(ppo_network) # Prevents caching issues with notebooks
from ppo_network import PPONetwork

import ppo_wrapper
importlib.reload(ppo_wrapper) # Prevents caching issues with notebooks
from ppo_wrapper import PPOWrapper

import hp_optimizer
importlib.reload(hp_optimizer) # Prevents caching issues with notebooks
from hp_optimizer import HPOptimizer

In [35]:
# LunarLander environment
env_id = 'LunarLander-v2'
max_episode_steps = 1024
num_envs = 16

env_kwargs = {
    'id': env_id,
    'max_episode_steps': max_episode_steps,
}

# Create vectorized environment
envs_vector = SyncVectorEnv([lambda: gym.make(**env_kwargs)] * num_envs)
states, infos = envs_vector.reset()

In [36]:
# Policy-Value Network
input_dims = 8
output_dims = 4
shared_hidden_dims = [1024, 512, 256] # 128]
shared_norm = nn.LayerNorm
policy_hidden_dims = [256, 128] #, 64]
policy_norm = nn.LayerNorm
value_hidden_dims = [256, 128] #, 64]
value_norm = nn.LayerNorm
activation = nn.ReLU

network_kwargs = {
    'input_dims': input_dims,
    'output_dims': output_dims,
    'shared_hidden_dims': shared_hidden_dims,
    'shared_norm': shared_norm,
    'policy_hidden_dims': policy_hidden_dims,
    'policy_norm': policy_norm,
    'value_hidden_dims': value_hidden_dims,
    'value_norm': value_norm,
    'activation': activation,
}

network = PPONetwork(**network_kwargs)

# Test forward pass
now = time.time()
for _ in range(100):
    states_tensor = torch.tensor(states, dtype=torch.float32)
    policy, value = network(states_tensor)
    
    actions_dist = torch.distributions.Categorical(logits=policy)
    actions = actions_dist.sample().numpy()
    
    states, rewards, dones, truncateds, infos = envs_vector.step(actions)
    #print(dones)

print(
    f'Elapsed time: per vectorized env: {(time.time() - now)/num_envs:.2f} s'
    )

/Users/axelsolhall/miniconda3/envs/PPOgym/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Elapsed time: per vectorized env: 0.02 s


In [37]:
# PPO Wrapper
lr = 3e-4
final_lr = 5e-6

gamma = 0.995
lam = 0.99

clip_eps = 0.25
final_clip_eps = 0.01

value_coef = 0.7

entropy_coef = 0.1
final_entropy_coef = 0.025

batch_size = 256
batch_epochs = 8
batch_shuffle = True
seperate_envs_shuffle = True

iterations = 2048

reward_normalize = True
truncated_reward = -300

debug_prints = False

ppo_kwargs = {
    'num_envs': num_envs,
    'lr': lr,
    'final_lr': final_lr,
    'gamma': gamma,
    'lam': lam,
    'clip_eps': clip_eps,
    'final_clip_eps': final_clip_eps,
    'value_coef': value_coef,
    'entropy_coef': entropy_coef,
    'final_entropy_coef': final_entropy_coef,
    'batch_size': batch_size,
    'batch_epochs': batch_epochs,
    'batch_shuffle': batch_shuffle,
    'seperate_envs_shuffle': seperate_envs_shuffle,
    'iterations': iterations,
    'reward_normalize': reward_normalize,
    'truncated_reward': truncated_reward,
    'debug_prints': debug_prints,   
}

ppo_wrapper = PPOWrapper(envs_vector, network, **ppo_kwargs)

now = time.time()
ppo_wrapper.train(generations=5)
after = time.time()
print(f'Elapsed time: {after - now:.2f} s')

### 5 generations ###
# default (cpu): 71.90 s

RuntimeError: Placeholder storage has not been allocated on MPS device!

In [23]:
# Hyperparameter optimization
hp_optimizer = HPOptimizer(
    env_kwargs=env_kwargs,
    num_envs=num_envs,
    network_class = PPONetwork,
    network_kwargs=network_kwargs,
    ppo_class=PPOWrapper,
    ppo_kwargs=ppo_kwargs,
)

parameters = [
    #'batch_shuffle',
    'reward_normalize',
    'seperate_envs_shuffle',
    #('gamma', [0.99, 0.995, 0.999]), 
    ('lam', [0.95, 0.975, 0.99]),
    #('final_clip_eps', [0.002, 0.01, 0.05]),
    ('final_lr', [1e-5, 1e-6, 1e-7]),
    ('batch_size', [64, 128, 256, 512]),
    ('batch_epochs', [2, 4, 8, 16]),
    ]

# evolutions = hp_optimizer.optimize_hyperparameters(
#     parameters, generations=40, num_trials = 16,
#     )


hp_optimizer.evolution_video(
    generations=10, video_folder = 'videos', increments=10, max_frames=max_episode_steps,
    )

Using device: mps
Running evolution with save generations: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


RuntimeError: Placeholder storage has not been allocated on MPS device!